# Let's explore the RFX MDSplus tree

## Importing and setting up stuff

In [5]:
import MDSplus as mds
import numpy as np
import matplotlib.pyplot as plt
import sys, os, time, random
from tqdm import tqdm
import h5py as h5
print(f'Python version: {sys.version}')
print(f'MDSplus version: {mds.__version__}')
np.set_printoptions(precision=3, suppress=True)

Python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
MDSplus version: 1.0.0


In [6]:
#color the terminal output
def pick_random_color():
    return '\033[38;5;{}m'.format(random.randint(8, 230))
ENDC = '\033[0m'
ERR = '\033[91m'+ 'ERR: '
OK = '\033[92m' 
WARN = '\033[93m'+ 'WARN: '

In [7]:
# define the shot number and tree
SHOT = 30810
rfx = mds.Tree('rfx', SHOT, 'readonly') # open the tree read-only
from convert_to_hdf5 import SEG_FAULT_NODES

## Traversing the tree

In [8]:
# traverse the tree, use MAX_DEPTH to limit the depth of the tree to traverse
# othwerwise the script will run for about 10 minutes
MAX_DEPTH = 13 # maximum depth of the tree to traverse
COLORS = [pick_random_color() for _ in range(MAX_DEPTH)]

In [ ]:
usage_depth, usage_breadth = {},{}
total_nodes_depth, total_nodes_breadth = [],[]

def traverse_tree_depth_first(max_depth, node, level=0, path='', node_type='child'):
    try: 
        if level >= max_depth: return # stop if the maximum depth is reached
        if node.getFullPath() in SEG_FAULT_NODES: return # skip the nodes that cause segfault
        if node_type == 'child': node_name = node.node_name.upper()
        elif node_type == 'member': node_name = node.node_name.lower()
        else: raise
        path = path + '/' + COLORS[level] + node_name + ENDC # add the node name
        total_nodes_depth.append(node) # add the node to the list
        print(f'{path}:{node.decompile()}') 
        # get the usage/type of the node
        try: usage_depth[str(node.usage)] += 1
        except: usage_depth[str(node.usage)] = 1
        # go through the children and members of the node
        for child in node.getChildren(): # get the children of the node
            traverse_tree_depth_first(max_depth, child, level + 1, path, 'child')
        for member in node.getMembers(): # get the members of the node
            traverse_tree_depth_first(max_depth, member, level + 1, path, 'member')
    except Exception as e:
        print(path + 'ERR:' + str(e))
        pass

# do the same but without recursion
def traverse_tree_breadth_first(max_depth, head_node):
    curr_nodes = [head_node]
    for d in range(max_depth):
        print('Depth:', d)
        next_nodes = []
        for node in curr_nodes:
            try:
                if node.getFullPath() in SEG_FAULT_NODES: continue # skip the nodes that cause segfault
                preprint = COLORS[d] + "   " * d + node.node_name + ENDC
                print(f'{preprint}:{node.decompile()}') # print the node
                total_nodes_breadth.append(node) # add the node to the list
                # get the usage/type of the node
                try: usage_breadth[str(node.usage)] += 1
                except: usage_breadth[str(node.usage)] = 1
                # get the children of the node
                for child in node.getChildren():
                    next_nodes.append(child)
                # get the members of the node
                for member in node.getMembers():
                    next_nodes.append(member)
            except: pass
        curr_nodes = next_nodes
        
# get the top node of the tree
head_node = rfx.getNode('\\TOP.RFX.MHD') # get the top node
# # test the functions, uncomment to run
traverse_tree_depth_first(MAX_DEPTH, head_node) # traverse the tree depth-first
traverse_tree_breadth_first(MAX_DEPTH, head_node) # traverse the tree breadth-first

print(f'Total nodes depth: {len(total_nodes_depth)}') # 96771, 96750
print(f'Total nodes breadth: {len(total_nodes_breadth)}') # 96771, 96750

In [ ]:
print(f'Usage depth: {usage_depth}')
print(f'Usage breadth: {usage_breadth}')

previous cell full depth: 'STRUCTURE': 8776, 'SUBTREE': 78, 'DEVICE': 642, 'ACTION': 1098, 'NUMERIC': 47760, 'TEXT': 17269, 'SIGNAL': 20904, 'ANY': 29, 'AXIS': 215

In [ ]:
print(f'top nodes: {[n.node_name for n in head_node.getChildren()]}')

## Exploring Signals

In [ ]:
search_space = '\\TOP.RFX.MHD.***' # *** means all nodes at this level
# search_space = '\\TOP.RFX.EDA.***' # * means all nodes at this level
# search_space = '\\TOP.RFX.***' # whole rfx tree
signal_nodes = rfx.getNodeWild(search_space, 'Signal') # get all nodes with the name 'Signal'
print(f'Found {len(signal_nodes)} of the type Signal in the search space {search_space}')

In [ ]:
# filter out the nodes without the data
data_signals = []
for node in tqdm(signal_nodes, leave=False):
    try: data = node.data(); data_signals.append(node)
    except: pass
print(f'Found {len(data_signals)}/{len(signal_nodes)} signals with data')

In [ ]:
# keep only the signals with raw data
raw_signals = []
for node in tqdm(signal_nodes, leave=False):
    try: data = node.raw_of().data(); raw_signals.append(node)
    except: pass
print(f'Found {len(raw_signals)}/{len(data_signals)} signals with raw data')

In [ ]:
# extract data from the signals and plot them
MAX_LOAD = 10 #10 #np.inf
MAX_LOAD = min(MAX_LOAD, len(raw_signals))
# select MAX_LOAD random signals
signals = random.sample(raw_signals, MAX_LOAD)
for node in (signals):
    signal = node.data()
    times = node.dim_of().data()
    unit = node.getUnits()
    full_path = node.getFullPath()
    try: node_help = node.getHelp()
    except: node_help = ''
    if signal.shape != times.shape:
        print(f'{full_path} has mismatched signal and time shapes')
        continue
    # plot the signal
    plt.figure()
    plt.plot(times, signal)
    plt.title(f'{full_path} [{unit}]\n{node_help}')
    plt.xlabel('Time [s]')
    plt.ylabel('Signal')
    plt.show()

## Exploring Text

In [ ]:
text_nodes = rfx.getNodeWild(search_space, 'Text') # get all the 'TEXT' nodes
print(f'Found {len(text_nodes)} of the type Text in the search space {search_space}')
# print all the text nodes
for node in text_nodes:
    try: print(f'{node.getFullPath()}={node.data()}')
    except: pass

## Exploring Times

In [11]:
times_diff_stds = []
times_lengths = []
def traverse_get_times(max_depth, head_node):
    curr_nodes = [head_node]
    for d in range(max_depth):
        print(f'Depth:{d}, nodes:{len(curr_nodes)}')
        next_nodes = []
        for node in curr_nodes:
            if node.getFullPath() in SEG_FAULT_NODES: continue # skip the nodes that cause segfault
            try: #get the times of the node
                times = node.dim_of().data()
                try :
                    assert times.ndim == 1
                    assert len(times) > 1, f'len(times)={len(times)}'
                    times_diff_stds.append(np.std(np.diff(times)))
                    times_lengths.append(len(times))
                except Exception as e:
                    print(f'{node.getFullPath()} ERR: {e}')
            except: pass
            try: # get the children of the node
                for child in node.getChildren(): next_nodes.append(child)
            except: pass
            try:# get the members of the node
                for member in node.getMembers(): next_nodes.append(member)
            except: pass
        curr_nodes = next_nodes

traverse_get_times(MAX_DEPTH, rfx.getNode('\\TOP.RFX.MHD')) # get the top node)

#plot a histogram of the times lenghts and a histogram of the times stds
lengths = np.array(times_lengths)
stds = np.array(times_diff_stds)

mean_stds = np.mean(stds)
std_stds = np.std(stds)
mean_lengths = np.mean(lengths)
std_lengths = np.std(lengths)

print(f'Mean stds: {mean_stds}, std stds: {std_stds}')
print(f'Mean lengths: {mean_lengths}, std lengths: {std_lengths}')



Depth:0, nodes:1
Depth:1, nodes:3
Depth:2, nodes:11
Depth:3, nodes:29
Depth:4, nodes:908
Depth:5, nodes:9872
\RFX::TOP.RFX.MHD.MHD_AC:CPCI_1:TIMING:DIO2_1:REC_TIMES ERR: len(times)=0
\RFX::TOP.RFX.MHD.MHD_AC:CPCI_2:TIMING:DIO2_1:REC_TIMES ERR: len(times)=0
\RFX::TOP.RFX.MHD.MHD_BC:CPCI_1:TIMING:DIO2_1:REC_TIMES ERR: len(times)=0
\RFX::TOP.RFX.MHD.MHD_BC:CPCI_2:TIMING:DIO2_1:REC_TIMES ERR: len(times)=0
\RFX::TOP.RFX.MHD.MHD_BR:CPCI_1:TIMING:DIO2_1:REC_TIMES ERR: len(times)=0
Depth:6, nodes:7440
Depth:7, nodes:0
Depth:8, nodes:0
Depth:9, nodes:0
Depth:10, nodes:0
Depth:11, nodes:0
Depth:12, nodes:0
Mean stds: 6.173824026638106e-05, std stds: 0.00014958059959981008
Mean lengths: 10283.444629601576, std lengths: 14363.868857845046
